In [1]:
!pip install openmeteo-requests
!pip install retry-requests
!pip install requests-cache

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.9 MB/s eta 0:00:00


In [45]:
import requests
import pandas as pd

# --- Feriados ---
url = 'https://date.nager.at/api/v3/publicholidays/2024/BR'
feriados_api = requests.get(url).json() #API em .json
df_feriados = pd.DataFrame(feriados_api)
df_feriados['date'] = pd.to_datetime(df_feriados['date']).dt.date #convertendo o formato da data

import openmeteo_requests
import requests_cache
from retry_requests import retry

# --- Clima ---
cache_session = requests_cache.CachedSession('.cache', expire_after = -1) #Utilizando o psudo código disponibilizado pelo site da API
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": -22.9064,
	"longitude": -43.1822, #Código do RJ
	"start_date": "2024-01-01",
	"end_date": "2024-08-01", #Intervalo de datas
	"daily": ["weather_code", "temperature_2m_mean"], #código e temperatura média diária
	"timezone": "auto"
}

responses = openmeteo.weather_api(url, params=params)

daily = responses[0].Daily()
daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()
daily_weather_code = daily.Variables(0).ValuesAsNumpy().astype(int).astype(str)

#Criando o data frame com as datas do intervalo de datas
daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_mean"] = daily_temperature_2m_mean #adicionando temperatura diária
daily_data["weather_code"] = daily_weather_code #Adicionando código do clima

df_clima = pd.DataFrame(data = daily_data)
df_clima['date'] = pd.to_datetime(df_clima['date']).dt.date #convertendo o formato da data para remover as horas, minutos,...

In [46]:
#Pergunta 01
qtd_feriados = df_feriados['date'].count()
#qtd_feriados = df_feriados['name'].nunique() #caso queira saber quantos feriados distintos ocorrem em 2024 (embora o carnaval ocorra 2 dias ele seria contado uma vez como 'o feriado de carnaval', mas para efeito análitico isso atrapalharia)
print(f'Quantidade de feriados: {qtd_feriados}')

Quantidade de feriados: 14


In [47]:
#Pergunta 02
df_feriados['month'] = pd.to_datetime(df_feriados['date']).dt.month
qtd_feriados_mes = df_feriados['month'].value_counts().sort_values(ascending=False) #Empate quádruplo
qtd_feriados_mes

,count
month,
2,2
3,2
5,2
11,2
1,1
4,1
7,1
9,1
10,1


In [48]:
#Pergunta 03
df_feriados['weekday'] = pd.to_datetime(df_feriados['date']).dt.weekday
#df_feriados_semana = df_feriados[(df_feriados['weekday'] > 0) & (df_feriados['weekday'] < 5)]['date'].count() #filtro convencional
df_feriados_semana = df_feriados.query('weekday > 0 and weekday < 5')['date'].count() #descobri o 'query' e resolvi usar
print(f'Quantidade de feriados em dias de semana: {df_feriados_semana}')

Quantidade de feriados em dias de semana: 7


In [49]:
#Pergunta 04
df_clima['month'] = pd.to_datetime(df_clima['date']).dt.month
df_clima_mes = df_clima.groupby('month')['temperature_2m_mean'].mean().sort_index()
df_clima_mes

,temperature_2m_mean
month,
1,26.694233
2,27.212988
3,26.416479
4,25.169930
5,24.990740
6,22.648958
7,21.105591
8,21.225414


In [50]:
# --- Código clima ---
import json
url = 'https://gist.githubusercontent.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c/raw/descriptions.json'
response = requests.get(url)
data = json.loads(response.text)

df_clima_code = pd.DataFrame() #Criando um dataframe para salvar os códigos de clima

for key in data.keys(): #iterando sob as chaves do dicionário importado do GitHub
  description = pd.DataFrame(data[key].values())
  description['code'] = key
  df_clima_code = pd.concat([df_clima_code, description], ignore_index=True)

df_clima_code.drop_duplicates(subset='code', inplace=True) #Removendo duplicatas
df_clima = df_clima.merge(df_clima_code[['code', 'description']], left_on='weather_code', right_on='code', how='left') #Mesclando os dataframes pelo código do clima

In [51]:
#Pergunta 05
df_clima_mes_descricao = df_clima.groupby(['month', 'description'])['date'].count().reset_index(name='count') #criando um dataframe de contagem para cada mês e tipo de clima
df_clima_mes_descricao_max = df_clima_mes_descricao.loc[df_clima_mes_descricao.groupby('month')['count'].idxmax()] #Pegando o máximo em cada mês
df_clima_mes_descricao_max

,month,description,count
8,1,Rain,10
13,2,Light Drizzle,13
22,3,Light Drizzle,9
27,4,Cloudy,7
36,5,Light Drizzle,10
44,6,Mainly Sunny,12
46,7,Cloudy,8
55,8,Partly Cloudy,1


In [55]:
#Pergunta 06
df_clima_feriado = df_feriados.merge(df_clima, on='date', how='left')
df_clima_feriado[['date','localName','temperature_2m_mean','description']]

,date,localName,temperature_2m_mean,description
0,2024-01-01,Confraternização Universal,24.854582,Light Drizzle
1,2024-02-12,Carnaval,30.156668,Mainly Sunny
2,2024-02-13,Carnaval,30.531670,Light Drizzle
3,2024-03-29,Sexta-feira Santa,25.044168,Light Rain
4,2024-03-31,Domingo de Páscoa,24.531664,Drizzle
5,2024-04-21,Dia de Tiradentes,23.087919,Partly Cloudy
6,2024-05-01,Dia do Trabalhador,28.108751,Sunny
7,2024-05-30,Corpus Christi,20.779579,Light Drizzle
8,2024-07-09,Revolução Constitucionalista de 1932,21.487915,Drizzle
9,2024-09-07,Dia da Independência,NaN,NaN


In [75]:
#Pergunta 07
df_clima_feriado[(df_clima_feriado['temperature_2m_mean'] < 20) & (df_clima_feriado['description'].isin(["Rain", "Cloudy"]))][['date','localName','temperature_2m_mean','description']]
#Todos os feriados são aproveitáveis

,date,localName,temperature_2m_mean,description


In [76]:
df_clima_feriado[(df_clima_feriado['temperature_2m_mean'] > 20) & (df_clima_feriado['description'].str.contains("Sunny"))][['date','localName','temperature_2m_mean','description']]
#Carnaval o melhor feriado do ano!

,date,localName,temperature_2m_mean,description
1,2024-02-12,Carnaval,30.156668,Mainly Sunny
6,2024-05-01,Dia do Trabalhador,28.108751,Sunny
